In [ ]:
import pathlib
import os
import json
from datetime import timedelta
from urllib.request import urlopen
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
# for offline ploting
#from plotly.offline import plot, iplot, init_notebook_mode
#init_notebook_mode(connected=True)
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

In [ ]:
def Donnees(fichier,liste):
    df = pd.read_csv(fichier, sep=';', usecols=liste,low_memory=False,nrows=4000000)#,skiprows = lambda x : x > 0 and x <= 800000, nrows = 100000)
    return df

Importation des modules et fonctions complémentaires

In [ ]:
x=Donnees('../input/valeurs-foncieres/valeursfoncieres-2019.txt',["Code departement","Coordonnees"])
m_1 = folium.Map(location=[46.420319479373994, 2.2458911741992877], tiles='cartodbpositron', zoom_start=5)
z=[]
for i in range(len(x["Coordonnees"])):
    if (x["Code departement"][i]=='05'):
        z.append(str(x["Coordonnees"][i]).split(','))

HeatMap(data=z, radius=5).add_to(m_1)
m_1

Fonction assurant le chargement 

In [ ]:
dpt_full=gpd.read_file('../input/additionaldataforfrenchmotorclaims/departements-version-simplifiee.geojson')
dpt = dpt_full[["code","geometry"]].set_index("code")

Chargemenent des données géographiques des départments

In [ ]:
x=Donnees('../input/valeurs-foncieres/valeursfoncieres-2019.txt',["Code departement","Coordonnees"])
m_2 = folium.Map(location=[46.420319479373994, 2.2458911741992877], tiles='cartodbpositron', zoom_start=5)
nb_dpt=x["Code departement"].value_counts()

Choropleth(geo_data=dpt.__geo_interface__,
           data=nb_dpt,
           key_on="feature.id",
           fill_color='YlGnBu', 
           legend_name='Nombre de demandes '
          ).add_to(m_2)
m_2

Affichage sur la carte du nombre de demandes par département

In [ ]:
x=Donnees('../input/valeurs-foncieres/valeursfoncieres-2019.txt',["Code departement","Coordonnees"])
y=Donnees('../input/valeurs-foncieres/valeursfoncieres-2020.txt',["Code departement","Coordonnees"])
m_3 = folium.Map(location=[46.420319479373994, 2.2458911741992877], tiles='cartodbpositron', zoom_start=5)
nb_dpt1=x["Code departement"].value_counts()
nb_dpt2=y["Code departement"].value_counts()
nb_dpt=nb_dpt2-nb_dpt1

Choropleth(geo_data=dpt.__geo_interface__,
           data=nb_dpt,
           key_on="feature.id",
           fill_color='YlGnBu', 
           legend_name='Evolution des ventes (2020-2019)'
          ).add_to(m_3)
m_3

Affichage sur la carte de l'évolution nombre de demandes par département

In [ ]:
x=Donnees('../input/valeurs-foncieres/valeursfoncieres-2019.txt',["Code departement","Valeur fonciere"])
y = {'1':0}
prix = {'1':0}
compt=0
for i in range(len(x)):
    
    if(str(x["Code departement"][i]) in y.keys()):
        y[str(x["Code departement"][i])]+=1
        prix[str(x["Code departement"][i])]+=float(str(x["Valeur fonciere"][i]))
    
    else:
        y[str(x["Code departement"][i])]=0
        prix[str(x["Code departement"][i])]=float(str(x["Valeur fonciere"][i]))


Calcul de la moyenne des transactions en 2019

In [ ]:
x2=Donnees('../input/valeurs-foncieres/valeursfoncieres-2020.txt',["Valeur fonciere","Code departement"])
y2 = {'1':0}
prix2={'1':0}

compt=0
for i in range(len(x2)):
    if(str(x2["Code departement"][i]) in y2.keys()):
        y2[str(x2["Code departement"][i])]+=1
        prix2[str(x2["Code departement"][i])]+=float(str(x["Valeur fonciere"][i]).replace(',','.'))
    else:
        y2[str(x2["Code departement"][i])]=0
        prix2[str(x2["Code departement"][i])]=float(str(x["Valeur fonciere"][i]).replace(',','.'))


Calcul de la moyenne des transactions en 2020

In [ ]:
plt.plot(y.keys(), y.values(), 1.0, color='b')
plt.plot(y2.keys(), y2.values(), 1.0, color='r')

Affichage des deux données précédentes

In [ ]:
moy=prix.copy()
for i in y.keys() : 
    if(y[i]!=0):
        moy[i]=prix[i]/y[i]
moy2=prix2.copy()
for i in y2.keys() :
    if(y2[i]!=0):
        moy2[i]=prix2[i]/y2[i]


plt.plot(prix.keys(), prix.values(), 10.0, color='b')
plt.plot(prix2.keys(), prix2.values(), 1.0, color='r')

plt.plot(moy.keys(), moy.values(), 1.0, color='b')
plt.plot(moy2.keys(), moy2.values(), 1.0, color='r')
